Swiss Pairs Simulation in Python

We start with a simple pure-Python model of a tournament. For now, we're not going to worry about how to simulate multiple runs, much less about performance. We just want to get a basic model working, and then we can refine it.

We need a few imports. The random module for generating random numbers is crucial.

In [29]:
import random
from collections import Counter
from itertools import permutations, zip_longest

In [30]:
# Keep track of the scores - at the start of the tournament, everyone has 0 wins
scores = {p:0 for p in range(20)}
# Keep track of who has already played who
played = set()

We need to find opponents for each pair. To do that, we sort pairs by score (and randomly within pairs with the same score),
then for each pair in turn assign as opponent the next pair in the list that they have not already played.

In [31]:
def pair(scores, played):
    pairs = list(sorted(scores.keys()))
    def key(p):
        # Rank by score, and then randomly for pairs with the same score
        return scores[p], random.random()
    ranked = list(sorted(pairs, key=key))
    games = []
    print("Ranking:", ranked)
    while ranked:
        # Get each pair in turn
        p1 = ranked.pop(0)
        print("Matching", p1, end="... ")
        # Find an opponent - the first pair they haven't played yet
        for n, p2 in enumerate(ranked):
            game = (p1, p2)
            print(p2, end=", ")
            if game not in played:
                del ranked[n]
                games.append(game)
                played.add(game)
                print("Found", p2)
                break
        else:
            # Looks like p1 has played every remaining pair.
            # In theory, there may be backtracking solutions that fail less often.
            # Don't worry about this for now, as the official rules are probably better anyway.
            print()
            raise RuntimeError("Unable to find a game for pair {}".format(p1))
    
    return games

pair({p:0 for p in range(20)}, set())

Ranking: [16, 3, 18, 13, 5, 10, 12, 8, 15, 6, 17, 9, 19, 11, 14, 4, 1, 2, 0, 7]
Matching 16... 3, Found 3
Matching 18... 13, Found 13
Matching 5... 10, Found 10
Matching 12... 8, Found 8
Matching 15... 6, Found 6
Matching 17... 9, Found 9
Matching 19... 11, Found 11
Matching 14... 4, Found 4
Matching 1... 2, Found 2
Matching 0... 7, Found 7


[(16, 3),
 (18, 13),
 (5, 10),
 (12, 8),
 (15, 6),
 (17, 9),
 (19, 11),
 (14, 4),
 (1, 2),
 (0, 7)]

In [32]:
# From itertools recipes
def grouper(iterable, n, fillvalue=None):
    "Collect data into fixed-length chunks or blocks"
    # grouper('ABCDEFG', 3, 'x') --> ABC DEF Gxx"
    args = [iter(iterable)] * n
    return zip_longest(*args, fillvalue=fillvalue)

# A better pairing algorithm - choose the set of pairings (from all possible) that minimises the
# total absolute score difference
# Better in theory, but WAY too slow to be practical
def pair_unusably_slow(scores, played):
    print("Getting all pairings")
    pairings = [list(grouper(perm, 2)) for perm in permutations(scores.keys())]
    print("Got {} - trimming".format(len(pairings)))
    pairings = [p for p in pairings if not any(match in played for match in p)]
    print("Result {}".format(len(pairings)))
    def key(pairing):
        total = 0
        for p1, p2 in pairing:
            total += max(p1,p2) - min(p1, p2) # abs difference
        return total
    return min(pairings, key=key)

We need to simulate playing a game. All we care about is who wins, so we just return the winner. For now, we assume that the pair with the higher number always wins. So the expectation is that the simulation should result in a list of pairs in descending order.

In [33]:
def game(p1, p2):
    return max(p1, p2)

OK, we now have our procedures for working out the pairings, and playing games. For the simulation, we just do this a set number of times.

In [34]:
def swiss(num_pairs, num_rounds):
    if num_pairs % 2:
        raise ValueError("An even number of pairs is required (num_pairs={})".format(num_pairs))
    scores = {p:0 for p in range(num_pairs)}
    played = set()
    
    for board in range(num_rounds):
        print("Round {}: Pairings played: {} out of {}".format(board, len(played), num_pairs*(num_pairs-1)//2))
        try:
            games = pair(scores, played)
        except RuntimeError:
            from collections import defaultdict
            d = defaultdict(list)
            for p1, p2 in played:
                d[p1].append(p2)
            for k in sorted(d):
                print(k, "played", list(sorted(d[k])))
            raise
        for p1, p2 in games:
            winner = game(p1, p2)
            scores[winner] += 1
            
    return scores

Counter(swiss(10, 5)).most_common()

Round 0: Pairings played: 0 out of 45
Ranking: [1, 3, 8, 5, 9, 0, 4, 7, 2, 6]
Matching 1... 3, Found 3
Matching 8... 5, Found 5
Matching 9... 0, Found 0
Matching 4... 7, Found 7
Matching 2... 6, Found 6
Round 1: Pairings played: 5 out of 45
Ranking: [4, 0, 1, 2, 5, 7, 8, 3, 9, 6]
Matching 4... 0, Found 0
Matching 1... 2, Found 2
Matching 5... 7, Found 7
Matching 8... 3, Found 3
Matching 9... 6, Found 6
Round 2: Pairings played: 10 out of 45
Ranking: [5, 0, 1, 6, 4, 3, 2, 7, 8, 9]
Matching 5... 0, Found 0
Matching 1... 6, Found 6
Matching 4... 3, Found 3
Matching 2... 7, Found 7
Matching 8... 9, Found 9
Round 3: Pairings played: 15 out of 45
Ranking: [0, 1, 5, 3, 2, 4, 8, 6, 9, 7]
Matching 0... 1, Found 1
Matching 5... 3, Found 3
Matching 2... 4, Found 4
Matching 8... 6, Found 6
Matching 9... 7, Found 7
Round 4: Pairings played: 20 out of 45
Ranking: [0, 3, 1, 2, 6, 5, 8, 7, 4, 9]
Matching 0... 3, Found 3
Matching 1... 2, 6, 5, Found 5
Matching 2... 6, 8, Found 8
Matching 6... 7, Found 

[(9, 5),
 (7, 4),
 (8, 4),
 (4, 3),
 (5, 3),
 (3, 2),
 (6, 2),
 (1, 1),
 (2, 1),
 (0, 0)]